In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [0]:
import os
# os.environ['PATH'] #+= ':/usr/local/cuda/bin'
# os.chdir("C:")

In [21]:

print("current_directory" , os.getcwd())
from google.colab import files


# 출처: http://jybaek.tistory.com/686 [컴맹 : 진화의 시작]

current_directory /content


In [22]:
mnist = input_data.read_data_sets('../data/mnist', one_hot=True)
mb_size = 64
Z_dim = 100
X_dim = mnist.train.images.shape[1]
y_dim = mnist.train.labels.shape[1]
h_dim = 128

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [0]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [0]:
""" Discriminator Net model """
with tf.device("/gpu:0") :
  X = tf.placeholder(tf.float32, shape=[None, 784])
  y = tf.placeholder(tf.float32, shape=[None, y_dim])

  D_W1 = tf.Variable(xavier_init([X_dim + y_dim, h_dim]))
  D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

  D_W2 = tf.Variable(xavier_init([h_dim, 1]))
  D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [0]:
def discriminator(x, y):
    inputs = tf.concat(axis=1, values=[x, y])
    D_h1 = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit


""" Generator Net model """
with tf.device("/gpu:0") :
  Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

  G_W1 = tf.Variable(xavier_init([Z_dim + y_dim, h_dim]))
  G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

  G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
  G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


def generator(z, y):
    inputs = tf.concat(axis=1, values=[z, y])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


In [0]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [0]:
with tf.device("/gpu:0") :
  G_sample = generator(Z, y)
  D_real, D_logit_real = discriminator(X, y)
  D_fake, D_logit_fake = discriminator(G_sample, y)

  D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
  D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
  D_loss = D_loss_real + D_loss_fake
  G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

  D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
  G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)



In [33]:
from google.colab import files
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(10000):
    if it % 1000 == 0:
        n_sample = 16

        Z_sample = sample_Z(n_sample, Z_dim)
        y_sample = np.zeros(shape=[n_sample, y_dim])
        y_sample[:, 7] = 1

        samples = sess.run(G_sample, feed_dict={Z: Z_sample, y:y_sample})
        
        fig = plot(samples)
        
        #fig, ax = plt.subplots(1, n_sample, figsize=(n_sample, 1))

        #for a in range(n_sample):
            #ax[a].set_axis_off()
            #ax[a].imshow(np.reshape(samples[a], (28, 28)))
            
        plt.savefig('{}.png'.format(str(i).zfill(3)))
        file_name='{}.png'.format(str(i).zfill(3))
        files.download(file_name)
        i += 1
        
        plt.close(fig)

    X_mb, y_mb = mnist.train.next_batch(mb_size)

    Z_sample = sample_Z(mb_size, Z_dim)
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: Z_sample, y:y_mb})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: Z_sample, y:y_mb})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D_loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print("-----------------------------------")

Iter: 0
D_loss: 1.55
G_loss: 2.309
-----------------------------------
Iter: 1000
D_loss: 0.005066
G_loss: 8.152
-----------------------------------
Iter: 2000
D_loss: 0.009458
G_loss: 6.672
-----------------------------------
Iter: 3000
D_loss: 0.1111
G_loss: 5.356
-----------------------------------
Iter: 4000
D_loss: 0.1413
G_loss: 5.713
-----------------------------------
Iter: 5000
D_loss: 0.126
G_loss: 5.012
-----------------------------------
Iter: 6000
D_loss: 0.2915
G_loss: 4.43
-----------------------------------
Iter: 7000
D_loss: 0.3316
G_loss: 5.357
-----------------------------------
Iter: 8000
D_loss: 0.5104
G_loss: 3.881
-----------------------------------
Iter: 9000
D_loss: 0.4517
G_loss: 3.683
-----------------------------------
